# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [34]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [35]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,33.53,55,20,11.50,CA,1715117070
1,1,adamstown,-25.0660,-130.1015,71.22,78,59,9.15,PN,1715117205
2,2,atafu village,-8.5421,-172.5159,85.10,75,96,11.05,TK,1715117177
3,3,edinburgh of the seven seas,-37.0676,-12.3116,61.72,78,100,40.87,SH,1715117211
4,4,waitangi,-43.9535,-176.5597,48.81,82,20,1.01,NZ,1715116996


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [36]:
%%capture --no-display

# Configure the map plot
city_data_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["city", "Humidity"]
)
    
# Load the city_data_df DataFrame

# Display the map
city_data_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df [
    (city_data_df["Max Temp"] >= 65) &
    (city_data_df["Max Temp"] <= 95) &
    (city_data_df["Cloudiness"] == 0) &
    (city_data_df["Wind Speed"] < 8)
]
# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,al mindak,20.1588,41.2834,67.60,38,0,1.23,SA,1715117261
137,137,port hedland,-20.3167,118.5667,68.05,83,0,5.75,AU,1715117272
148,148,ar ruways,24.1103,52.7306,86.40,60,0,7.85,AE,1715117273
149,149,ormara,25.2088,64.6357,81.27,80,0,6.60,PK,1715117273
151,151,olinda,-8.0089,-34.8553,80.33,83,0,5.75,BR,1715117068
188,188,tilisarao,-32.7329,-65.2911,68.40,45,0,7.90,AR,1715117277
189,189,miandrivazo,-19.5167,45.4667,74.48,52,0,3.02,MG,1715117278
213,213,warrenton,-28.1140,24.8475,65.01,27,0,5.48,ZA,1715117281
229,229,vila velha,-20.3297,-40.2925,78.75,78,0,4.61,BR,1715117111
230,230,karratha,-20.7377,116.8463,75.65,69,0,3.98,AU,1715117208


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]                                                                 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
44,al mindak,SA,20.1588,41.2834,38,
137,port hedland,AU,-20.3167,118.5667,83,
148,ar ruways,AE,24.1103,52.7306,60,
149,ormara,PK,25.2088,64.6357,80,
151,olinda,BR,-8.0089,-34.8553,83,
188,tilisarao,AR,-32.7329,-65.2911,45,
189,miandrivazo,MG,-19.5167,45.4667,52,
213,warrenton,ZA,-28.1140,24.8475,27,
229,vila velha,BR,-20.3297,-40.2925,78,
230,karratha,AU,-20.7377,116.8463,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel

radius = 10000

params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    print( name_address.status_code)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
200
al mindak - nearest hotel: No hotel found
200
port hedland - nearest hotel: The Esplanade Hotel
200
ar ruways - nearest hotel: No hotel found
200
ormara - nearest hotel: No hotel found
200
olinda - nearest hotel: Fidji Motel
200
tilisarao - nearest hotel: No hotel found
200
miandrivazo - nearest hotel: Baobab
200
warrenton - nearest hotel: No hotel found
200
vila velha - nearest hotel: Hotel Prainha
200
karratha - nearest hotel: Karratha International Hotel
200
'alaqahdari dishu - nearest hotel: No hotel found
200
turabah - nearest hotel: No hotel found
200
tsiombe - nearest hotel: No hotel found
200
ad dilam - nearest hotel: No hotel found
200
valley center - nearest hotel: Valley View Casino and Hotel
200
port saint john's - nearest hotel: Outback Inn
200
al jawf - nearest hotel: No hotel found
200
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
200
vilhena - nearest hotel: No hotel found
200
nogales - nearest hotel: No hotel found
200

,City,Country,Lat,Lng,Humidity,Hotel Name
44,al mindak,SA,20.1588,41.2834,38,No hotel found
137,port hedland,AU,-20.3167,118.5667,83,The Esplanade Hotel
148,ar ruways,AE,24.1103,52.7306,60,No hotel found
149,ormara,PK,25.2088,64.6357,80,No hotel found
151,olinda,BR,-8.0089,-34.8553,83,Fidji Motel
188,tilisarao,AR,-32.7329,-65.2911,45,No hotel found
189,miandrivazo,MG,-19.5167,45.4667,52,Baobab
213,warrenton,ZA,-28.1140,24.8475,27,No hotel found
229,vila velha,BR,-20.3297,-40.2925,78,Hotel Prainha
230,karratha,AU,-20.7377,116.8463,69,Karratha International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
# Removing cities without a hotel listed
hotel_df_simple = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df_simple

,City,Country,Lat,Lng,Humidity,Hotel Name
137,port hedland,AU,-20.3167,118.5667,83,The Esplanade Hotel
151,olinda,BR,-8.0089,-34.8553,83,Fidji Motel
189,miandrivazo,MG,-19.5167,45.4667,52,Baobab
229,vila velha,BR,-20.3297,-40.2925,78,Hotel Prainha
230,karratha,AU,-20.7377,116.8463,69,Karratha International Hotel
335,valley center,US,33.2184,-117.0342,57,Valley View Casino and Hotel
363,port saint john's,ZA,-31.6229,29.5448,49,Outback Inn
416,hotan,CN,37.0997,79.9269,26,Hua Yi International Hotel (accepts foreigners)
506,piti village,GU,13.4626,144.6933,81,The Cliff Hotel


In [41]:
%%capture --no-display

# Configure the map plot

complete_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["city", "Humidity", "Hotel Name", "Country"]
)
    
# Display the map
complete_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)